POC: Road detection using VLP-16
=============================
___
**Author:** *Thomas Paul*, *Nagarjuna Vemuri*

**Version:** *1.0*
___

<center>
    <video  width="800" height="450" controls src="resources/output.mp4" />
</center>


The objective of this work is to detect road from a sparse point cloud generated from Velodyne VLP-16 Puck LITE<sup>[1]</sup> which is a low cost, lower resolution LiDAR. In our earlier work, we examined solutions for detecting road from a dense point cloud generated from high resolution, high-end LiDAR- Velodyne HDL-64E<sup>[2]</sup>.

### prerequisite
As this work is an extension, we urge you to go through the documentation of phase one to get an introduction to this work. 

### Overview

Through this work, we discovered that the LoDNN<sup>[3]</sup> model which we used to detect road from dense point cloud performs well, with slight modifications for processing sparse point cloud. The point cloud generated from VLP-16 is transformed to a common coordinate system. Then features are extracted from the transformed point cloud and encoded in gray scale images. These images are fed into LoDNN to get a confidence map which is subjected to thresholding to get an optimal heatmap.

### Dataset

The dataset for this work was created in-house as there was no openly available dataset. Three dataset ros bags and a calibration ros bag was created by driving around the campus in the golf cart with VLP-16 mounted on it. The bag details are as follows.

|Bag name 	|File name                  |Total duration(s)	|Start-end time(s)	|Start-end timestamp   	|Total frames   	|Selected frames 	|Number of samples|Category|
|---     	|---	                    |---	        |---	        |---	|---	|---	|---	|---	|
|Bag 1   	|2018-10-15-16-48-34.bag   	|284.452389   	|15-279     	|1539602330.222308 - 1539602594.234182   	|2857  	|2663   	   	|267|Train|
|Bag 2   	|2018-10-15-16-54-08.bag   	|206.064596   	|11-199   	    |1539602659.840781 - 1539602847.80224   	|2070  	|1890   	|189|Test   	|
|Bag 3   	|2018-10-15-16-57-39.bag   	|374.989108   	|10-373   	    |1539602869.530959 - 1539603232.475730   	|3767  	|3648   	|365|Train  	|


Transformation values- yaw, pitch, roll and x, y, z translations were found using calibration bag. 821 samples were selected from three dataset bags and were annotated. The training set which contains 632 samples was created from bag one and bag three. The test set was created by taking 189 samples from bag 2.

Calibration and annotation is discussed in following sections

### Calibration

In calibration process, we need to find transformation values- yaw, pitch, roll and x, y, z translations. These transformation values are used to transform the point cloud generated by LiDAR to a common frame. This process has to be done maunally everytime before doing the road detection. 

![](resources/raw_pc.png "")

Above is the visualization of point cloud generated by VLP-16. As we can see, some points are below the grid. We have to find transformation values such that the above point cloud starts from the grid- ie, the road points should be on the grid. See below picture.

![](resources/transformed_pc.png "")

The transformation values can be found through trail and error. Below are the steps.

1. Start `detect_road.py` from `road_detection` package.
2. In rviz suscribe to topic `/transformed_pc` for visualzing the transformed point cloud.
3. Publish the transormation values by running `rostopic pub /dof_ip std_msgs/String "data: '0,0,0,0,0,0'"`. The order of the values given in data is yaw, pitch, roll, x, y, z.
4. Adjust yaw, pitch, roll, x, y, z by performing step 3, such that road in point cloud sits on top of the grid like in the above figure.

### Filtering

Before processing for features, the unstructured point cloud is filtered. Following are the conditions on which a point is selected.
+ $\text{x} \in [2.0, 27.0]$
+ $\text{y} \in [−10.0, 10.0]$
+ $\text{z} \in [−1.0, 2.6]$

As you can see from the above, the points which are in 10 meters on either side of the golf cart and in front of the cart within a range of 2 to 27 meters and with height of -1 to 2.6 meters are taken. 

### Annotation

For annotation, point cloud is transformed to top down view(brid's eye view) images with a grid resolution of 0.1 meter. Then, these images are labeled using `labelme`<sup>[4]</sup> tool. JSON files are generated by labelme after drawing polygons to classify roads on BEV images. 

Scripts and files to generate instance segmentation masks are located at `source/annotation/tool`.
The json files and BEV images are put into a directory named `annotations`. Then `./labelme2voc.py labels.txt annotations/ output/` command is ran from parent diredctory to create instance segmentation mask PNGs which wil be created at `output/SegmentationClassPNG`.


### Features

The features are extracted from point cloud and encoded into top down view gray scale images with a resolution of 0.1 meter. The features- count, mean reflectance, and, maximum, minimum, mean and standard deviation of elevation that are specified in LoDNN are extracted. We have considered one more feature- unique count of rings per grid<sup>[5]</sup>. The shape of the feature array for a point cloud is `(250, 200, 7)`. This is given as the input to the CNN network.

The implementation of extracting features is `get_features()` at `source/notebooks/pipeline.ipynb`.

### Training

Training is done using `_train()` at `source/notebooks/pipeline.ipynb`. The model used is LoDNN with modified input shape and deconvolution instead of max unpooling layer. The implementation is at `source/notebooks/models_src/LoDNN.py`. 

The configuration used for the training is as below.
+ validation split: 0.1
+ loss_fn: binary_crossentropy
+ batch_size: 4
+ epochs: 1000
+ learning_rate: 0.0001
+ Optimiser: Adam
+ Dataset:
    + Training - Bag one and three. Annotation version 2.
    + Test - Bag two. Annotation version 3.

See the training implementation [here](training pipeline.html).

### Post processing

To fine tune the accuracy, we have applied thresholding on maximum elevation per grid and unique number of rings per grid. All grids which has maximum elevation more than 0.4 meters is marked as non-road. Any grid which has unique number of rings more than 1 is marked as non-road. This was done to reduce false positives. We observed that post processing increased the model accuracy slightly by 0.002% on test dataset.

### ROS Implementation

After model development, the solution was ported to ROS Indigo as a package- `road_detection`. The implementation is at `source/rosws/src/road_detection/detect_road.py`. The dependencies are as follows.
+ tensorflow-gpu==1.4.0
+ Keras==2.1.4
+ numpy==1.15.4
+ CUDA==8.0.61

See the ROS implementation [here](Road detection ros package.html).

### Result
After training multiple models, we decided to take model `2018_12_06_21_15_34` to deploy it wih ROS code. The model benchmark is as follows,
+ Accuracy:
    + Test - 95.90%
    + Train - 98.43%
+ Execution time:
    + Feature extration: 2.66 ms ± 38.6 µs
    + Model prediction(with out model optimisation) with thresholding: 10.7 ms ± 119 µs
    + ROS code: 50 ms ± 10 ms 

### Further work

+ Using less resolution for the grid while taking the BEV. In this work we have used a resolution of 0.1 meters. This resolution worked very well in phase one. But in this work, we found that the difference between the maximum and minimum elevation in grid of bicycle and pedestrian is very negligible and even comparable to that of road! This is because there are very few poinsts per grid because of the smaller grid size. So reducing the resolution and repeating this work might increase the model accuracy. We suggest an exploratory work with resolution of 0.4 or 0.5 for taking top view.
+ Another way to increase the model accuracy is to add more better annotated data. In this work, we had difficulty in annotating LiDAR data and the work can be enhanced by having a annotation tool with which we can add more data.
+ When we reduced the input size to the model, we saw that there was a drastic reduction in LoDNN accuracy. An exploratory work with lower size input for the network would be worthwhile.
+ While doing demo of the work, we will most likely spend more time in calibration. A calibration guideline for sensors including LiDAR would help developers focus on the problem rather than calibration. Calibration process guideline along with a tool to automate the calibration of LiDAR and possibly other sensors would be a great avenue of research.

### References
+ [[1]](https://velodynelidar.com/vlp-16-lite.html) Velodyne VLP-16 Puck LITE
+ [[2]](https://velodynelidar.com/hdl-64e.html) Velodyne HDL-64E
+ [[3]](https://arxiv.org/pdf/1703.03613.pdf) Fast LIDAR-based Road Detection Using Fully Convolutional Neural Networks. (LoDNN)
+ [[4]](https://github.com/wkentaro/labelme) Labelme tool for instance segmentation
+ [[5]](https://stackoverflow.com/q/53513497/6561141) Numpy: Finding count of distinct values from associations through binning

### Acknowledgements
We thank Nahas Pareekutty for his constant mentoring- his help and support was very crucial for the work. We express my sincere gratitude towards Dr. Sreekanta Gupta and Saurabh Mishra for their support.

We thank Mohan Kamatam, Navin Rawther, and, Aravindh K  for their encouragement and perspectives.